# Praca domowa 2

In [1]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import math

In [2]:
data = pd.read_csv('diabetic_data_2.csv')

In [3]:
data.head(10)

,gender,age,discharge_disposition_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,diag_1_Other,diag_1_Respiratory,max_glu_serum_>200,max_glu_serum_>300,max_glu_serum_None,max_glu_serum_Norm,A1Cresult_>7,A1Cresult_>8,A1Cresult_None,A1Cresult_Norm
0,0,15,1,3,59,0,18,0,0,0,...,1,0,0,0,1,0,0,0,1,0
1,0,25,1,2,11,5,13,2,0,1,...,1,0,0,0,1,0,0,0,1,0
2,1,35,1,2,44,1,16,0,0,0,...,1,0,0,0,1,0,0,0,1,0
3,1,45,1,1,51,0,8,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,1,55,1,3,31,6,16,0,0,0,...,0,0,0,0,1,0,0,0,1,0
5,1,65,1,4,70,1,21,0,0,0,...,0,0,0,0,1,0,0,0,1,0
6,1,75,1,5,73,0,12,0,0,0,...,0,0,0,0,1,0,0,0,1,0
7,0,85,1,13,68,2,28,0,0,0,...,0,0,0,0,1,0,0,0,1,0
8,0,95,0,12,33,3,18,0,0,0,...,0,0,0,0,1,0,0,0,1,0
9,0,45,1,9,47,2,17,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [4]:
data.shape

(98052, 56)

W pierwszej pracy domowej pozbyłem się rekordów które były kolejnymi wizytami tych pacjentów zostawiając tylko pierwszą z wizyt. Usunąłem także osoby które zmarły. W wyniku tych działań zmniejszyłem liczbę rekordów do około 68 tys. Jednak jak zauważyłem wpłynęło to negatywnie na mój model dlatego zdecydowałem się przywrócić tych pacjentów.

In [5]:
df = data.copy()

y = df['readmitted'].copy()
df.drop('readmitted', axis=1, inplace=True)
X = df.copy()

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 21, stratify = y)

## Trening z domyślnymi parametrami (default)

In [11]:
from sklearn.ensemble import ExtraTreesClassifier

In [8]:
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier()
clf.fit(X_train, y_train)

ExtraTreesClassifier()

In [9]:
proba = clf.predict_proba(X_test)[:, 1]
pred = clf.predict(X_test)

In [10]:
from sklearn.metrics import roc_auc_score
roc = "{:.5f}".format(roc_auc_score(y_test, proba))
print(f"Współczynnik roc_auc wynosi: {roc}")

Współczynnik roc_auc wynosi: 0.64464


In [11]:
from sklearn.metrics import accuracy_score
acc = "{:.5f}".format(accuracy_score(y_test, pred))
print(f"Współczynnik accuracy wynosi: {acc}")

Współczynnik accuracy wynosi: 0.60996


## Trening z ręcznie zmienianymi parametrami

In [12]:
clf = ExtraTreesClassifier(criterion = 'gini', n_estimators = 200, max_features = 40, min_samples_leaf=100)
clf.fit(X_train, y_train)

ExtraTreesClassifier(max_features=40, min_samples_leaf=100, n_estimators=200)

In [13]:
proba = clf.predict_proba(X_test)[:, 1]
pred = clf.predict(X_test)

In [14]:
roc = "{:.5f}".format(roc_auc_score(y_test, proba))
print(f"Współczynnik roc_auc wynosi: {roc}")

Współczynnik roc_auc wynosi: 0.67811


In [15]:
acc = "{:.5f}".format(accuracy_score(y_test, pred))
print(f"Współczynnik accuracy wynosi: {acc}")

Współczynnik accuracy wynosi: 0.63082


## Grid Search

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
clf = ExtraTreesClassifier()
params = {
    'criterion': ['gini', 'entropy'],
    'n_estimators': [100, 200, 500],
    'min_samples_leaf': [5, 50, 100],
    'max_features' : [40, 'auto']
}

gs = GridSearchCV(estimator = clf, param_grid = params, n_jobs = -1, verbose = 2, cv = 3,
                  scoring='roc_auc')
gs.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 10.2min finished


GridSearchCV(cv=3, estimator=ExtraTreesClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': [40, 'auto'],
                         'min_samples_leaf': [5, 50, 100],
                         'n_estimators': [100, 200, 500]},
             scoring='roc_auc', verbose=2)

In [18]:
gs.best_params_

{'criterion': 'gini',
 'max_features': 40,
 'min_samples_leaf': 50,
 'n_estimators': 200}

In [19]:
gs = gs.best_estimator_

In [22]:
pred = gs.predict(X_test)
proba = gs.predict_proba(X_test)[:, 1]

In [23]:
roc = "{:.5f}".format(roc_auc_score(y_test, proba))
print(f"Współczynnik roc_auc wynosi: {roc}")

Współczynnik roc_auc wynosi: 0.67932


In [24]:
acc = "{:.5f}".format(accuracy_score(y_test, pred))
print(f"Współczynnik accuracy wynosi: {acc}")

Współczynnik accuracy wynosi: 0.63041


## Random Search

In [25]:
from sklearn.model_selection import RandomizedSearchCV

In [26]:
clf = ExtraTreesClassifier()
params = {
    'criterion': ['gini', 'entropy'],
    'n_estimators': np.arange(100, 501, 50, dtype=int),
    'min_samples_leaf': np.arange(5, 101, 10, dtype=int),
    'max_features' : np.arange(5, 50, 5, dtype=int)
}

rs = RandomizedSearchCV(estimator = clf, param_distributions = params, verbose=2, 
                               cv = 3, n_iter = 50, n_jobs = -1, scoring='roc_auc')
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   44.5s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 16.1min finished


RandomizedSearchCV(cv=3, estimator=ExtraTreesClassifier(), n_iter=50, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_features': array([ 5, 10, 15, 20, 25, 30, 35, 40, 45]),
                                        'min_samples_leaf': array([ 5, 15, 25, 35, 45, 55, 65, 75, 85, 95]),
                                        'n_estimators': array([100, 150, 200, 250, 300, 350, 400, 450, 500])},
                   scoring='roc_auc', verbose=2)

In [27]:
rs.best_params_

{'n_estimators': 450,
 'min_samples_leaf': 25,
 'max_features': 20,
 'criterion': 'entropy'}

In [28]:
rs = rs.best_estimator_

In [29]:
pred = rs.predict(X_test)
proba = rs.predict_proba(X_test)[:, 1]

In [30]:
roc = "{:.5f}".format(roc_auc_score(y_test, proba))
print(f"Współczynnik roc_auc wynosi: {roc}")

Współczynnik roc_auc wynosi: 0.68068


In [31]:
acc = "{:.5f}".format(accuracy_score(y_test, pred))
print(f"Współczynnik accuracy wynosi: {acc}")

Współczynnik accuracy wynosi: 0.63179


## Optymalizacja Bayesowska

In [8]:
from skopt import BayesSearchCV

In [9]:
params = {
    'criterion': ['gini', 'entropy'],
    'n_estimators': np.arange(100, 501, 50, dtype=int),
    'min_samples_leaf': np.arange(5, 101, 10, dtype=int),
    'max_features' : np.arange(5, 50, 5, dtype=int)
}

In [13]:
clf = ExtraTreesClassifier()
bayes = BayesSearchCV(
        estimator=clf,
        search_spaces=params,
        scoring='roc_auc',
        n_jobs=-1,
        n_iter=50,
        verbose=0,
        random_state=21,
        cv=3
    )
bayes.fit(X_train, y_train)
bayes.best_params_

OrderedDict([('criterion', 'gini'),
             ('max_features', 20),
             ('min_samples_leaf', 35),
             ('n_estimators', 450)])

In [14]:
bayes.best_params_

OrderedDict([('criterion', 'gini'),
             ('max_features', 20),
             ('min_samples_leaf', 35),
             ('n_estimators', 450)])

In [15]:
bayes = bayes.best_estimator_

In [16]:
pred = bayes.predict(X_test)
proba = bayes.predict_proba(X_test)[:, 1]

In [19]:
roc = "{:.5f}".format(roc_auc_score(y_test, proba))
print(f"Współczynnik roc_auc wynosi: {roc}")

Współczynnik roc_auc wynosi: 0.68039


In [20]:
acc = "{:.5f}".format(accuracy_score(y_test, pred))
print(f"Współczynnik accuracy wynosi: {acc}")

Współczynnik accuracy wynosi: 0.63082
